<a href="https://colab.research.google.com/github/AkarshMSonni/fetch/blob/main/Fetch_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Q1. Inspect the unstructured JSON data and draw an ER Diagram

Converting unstructured data to a dataframe to review

In [1]:
#Importing libraries
import pandas as pd
import json
from ast import literal_eval


Normalizing and reading the Receipts table

In [2]:
# Read the JSON file with one JSON object per line
with open('/content/drive/MyDrive/Fetch Sample Data/receipts.json') as f:
    lines = f.readlines()

# Normalize each JSON object and convert date columns
normalized_data = []
for line in lines:
    data = json.loads(line)
    df_receipts = pd.json_normalize(data)

    # Convert specific date columns to datetime format
    date_columns = ['createDate.$date', 'dateScanned.$date', 'finishedDate.$date', 'modifyDate.$date', 'pointsAwardedDate.$date', 'purchaseDate.$date']
    for column in date_columns:
        if column in df_receipts.columns:
            df_receipts[column] = pd.to_datetime(df_receipts[column]/1000, unit='s')

    normalized_data.append(df_receipts)

# Concatenate all normalized DataFrames
df_receipts = pd.concat(normalized_data, ignore_index=True)

# Print the resulting DataFrame
df_receipts.head(5)


,bonusPointsEarned,bonusPointsEarnedReason,pointsEarned,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId,_id.$oid,createDate.$date,dateScanned.$date,finishedDate.$date,modifyDate.$date,pointsAwardedDate.$date,purchaseDate.$date
0,500.0,"Receipt number 2 completed, bonus point schedu...",500.0,5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6,5ff1e1eb0a720f0523000575,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,2021-01-03 00:00:00
1,150.0,"Receipt number 5 completed, bonus point schedu...",150.0,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052,5ff1e1bb0a720f052300056b,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,2021-01-02 15:24:43
2,5.0,All-receipts receipt bonus,5,1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b,5ff1e1f10a720f052300057a,2021-01-03 15:25:37,2021-01-03 15:25:37,NaT,2021-01-03 15:25:42,NaT,2021-01-03 00:00:00
3,5.0,All-receipts receipt bonus,5.0,4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.00,5ff1e1eacfcf6c399c274ae6,5ff1e1ee0a7214ada100056f,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:39,2021-01-03 15:25:34,2021-01-03 00:00:00
4,5.0,All-receipts receipt bonus,5.0,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.00,5ff1e194b6a9d73a3a9f1052,5ff1e1d20a7214ada1000561,2021-01-03 15:25:06,2021-01-03 15:25:06,2021-01-03 15:25:11,2021-01-03 15:25:11,2021-01-03 15:25:06,2021-01-02 15:25:06


Need to further normalize the rewardsReceiptItemList column

In [3]:
df_receipts = df_receipts.explode('rewardsReceiptItemList')
df_receipts.reset_index(inplace=True)

In [4]:
df_receipts = df_receipts.fillna({'rewardsReceiptItemList':'{}'})
df_receipts['rewardsReceiptItemList'] = df_receipts['rewardsReceiptItemList'].apply(lambda x:str(x))
df_receipts['rewardsReceiptItemList'] = df_receipts['rewardsReceiptItemList'].apply(literal_eval)

In [5]:
df_receipts.head(5)

,index,bonusPointsEarned,bonusPointsEarnedReason,pointsEarned,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId,_id.$oid,createDate.$date,dateScanned.$date,finishedDate.$date,modifyDate.$date,pointsAwardedDate.$date,purchaseDate.$date
0,0,500.0,"Receipt number 2 completed, bonus point schedu...",500.0,5.0,"{'barcode': '4011', 'description': 'ITEM NOT F...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6,5ff1e1eb0a720f0523000575,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,2021-01-03 00:00:00
1,1,150.0,"Receipt number 5 completed, bonus point schedu...",150.0,2.0,"{'barcode': '4011', 'description': 'ITEM NOT F...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052,5ff1e1bb0a720f052300056b,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,2021-01-02 15:24:43
2,1,150.0,"Receipt number 5 completed, bonus point schedu...",150.0,2.0,"{'barcode': '028400642255', 'description': 'DO...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052,5ff1e1bb0a720f052300056b,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,2021-01-02 15:24:43
3,2,5.0,All-receipts receipt bonus,5,1.0,"{'needsFetchReview': False, 'partnerItemId': '...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b,5ff1e1f10a720f052300057a,2021-01-03 15:25:37,2021-01-03 15:25:37,NaT,2021-01-03 15:25:42,NaT,2021-01-03 00:00:00
4,3,5.0,All-receipts receipt bonus,5.0,4.0,"{'barcode': '4011', 'description': 'ITEM NOT F...",FINISHED,28.00,5ff1e1eacfcf6c399c274ae6,5ff1e1ee0a7214ada100056f,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:39,2021-01-03 15:25:34,2021-01-03 00:00:00


In [28]:
df_receipts_norm = pd.json_normalize(df_receipts['rewardsReceiptItemList'],errors='ignore',record_prefix='rewardsReceiptItemList')\
.add_prefix('rewardsReceiptItemList.')

df_receipts_norm.columns


Index(['rewardsReceiptItemList.barcode', 'rewardsReceiptItemList.description',
       'rewardsReceiptItemList.finalPrice', 'rewardsReceiptItemList.itemPrice',
       'rewardsReceiptItemList.needsFetchReview',
       'rewardsReceiptItemList.partnerItemId',
       'rewardsReceiptItemList.preventTargetGapPoints',
       'rewardsReceiptItemList.quantityPurchased',
       'rewardsReceiptItemList.userFlaggedBarcode',
       'rewardsReceiptItemList.userFlaggedNewItem',
       'rewardsReceiptItemList.userFlaggedPrice',
       'rewardsReceiptItemList.userFlaggedQuantity',
       'rewardsReceiptItemList.needsFetchReviewReason',
       'rewardsReceiptItemList.pointsNotAwardedReason',
       'rewardsReceiptItemList.pointsPayerId',
       'rewardsReceiptItemList.rewardsGroup',
       'rewardsReceiptItemList.rewardsProductPartnerId',
       'rewardsReceiptItemList.userFlaggedDescription',
       'rewardsReceiptItemList.originalMetaBriteBarcode',
       'rewardsReceiptItemList.originalMetaBriteDescri

Merging the exploded column back to the original receipts dataframe

In [49]:
df_receipts_final = pd.merge(df_receipts, df_receipts_norm, left_index=True, right_index=True, how='outer')
df_receipts_final['rewardsReceiptStatus'].unique()

array(['FINISHED', 'REJECTED', 'FLAGGED', 'SUBMITTED', 'PENDING'],
      dtype=object)

In [10]:
#User friendly names for the colum headers

cols = {'_id.$oid':'receipt_id',
        'createDate.$date':'createDate',
        'dateScanned.$date':'dateScanned',
        'finishedDate.$date':'finishedDate',
        'modifyDate.$date':'modifyDate',
        'pointsAwardedDate.$date':'pointsAwardedDate',
        'purchaseDate.$date':'purchaseDate'}

df_receipts_final.rename(columns=cols,
          inplace=True)

df_receipts_final.columns

Index(['index', 'bonusPointsEarned', 'bonusPointsEarnedReason', 'pointsEarned',
       'purchasedItemCount', 'rewardsReceiptItemList', 'rewardsReceiptStatus',
       'totalSpent', 'userId', 'receipt_id', 'createDate', 'dateScanned',
       'finishedDate', 'modifyDate', 'pointsAwardedDate', 'purchaseDate',
       'rewardsReceiptItemList.barcode', 'rewardsReceiptItemList.description',
       'rewardsReceiptItemList.finalPrice', 'rewardsReceiptItemList.itemPrice',
       'rewardsReceiptItemList.needsFetchReview',
       'rewardsReceiptItemList.partnerItemId',
       'rewardsReceiptItemList.preventTargetGapPoints',
       'rewardsReceiptItemList.quantityPurchased',
       'rewardsReceiptItemList.userFlaggedBarcode',
       'rewardsReceiptItemList.userFlaggedNewItem',
       'rewardsReceiptItemList.userFlaggedPrice',
       'rewardsReceiptItemList.userFlaggedQuantity',
       'rewardsReceiptItemList.needsFetchReviewReason',
       'rewardsReceiptItemList.pointsNotAwardedReason',
       'rew

Normalizing and reading the Brands table

In [31]:
# Read the JSON file with one JSON object per line
with open('/content/drive/MyDrive/Fetch Sample Data/brands.json') as f:
    lines = f.readlines()

# Normalize each JSON object and convert date columns
normalized_data = []
for line in lines:
    data = json.loads(line)
    df_brands = pd.json_normalize(data)
    normalized_data.append(df_brands)

# Concatenate all normalized DataFrames
df_brands = pd.concat(normalized_data, ignore_index=True)

# Print the resulting DataFrame
df_brands.head()


,barcode,category,categoryCode,name,topBrand,_id.$oid,cpg.$id.$oid,cpg.$ref,brandCode
0,511111019862,Baking,BAKING,test brand @1612366101024,False,601ac115be37ce2ead437551,601ac114be37ce2ead437550,Cogs,NaN
1,511111519928,Beverages,BEVERAGES,Starbucks,False,601c5460be37ce2ead43755f,5332f5fbe4b03c9a25efd0ba,Cogs,STARBUCKS
2,511111819905,Baking,BAKING,test brand @1612366146176,False,601ac142be37ce2ead43755d,601ac142be37ce2ead437559,Cogs,TEST BRANDCODE @1612366146176
3,511111519874,Baking,BAKING,test brand @1612366146051,False,601ac142be37ce2ead43755a,601ac142be37ce2ead437559,Cogs,TEST BRANDCODE @1612366146051
4,511111319917,Candy & Sweets,CANDY_AND_SWEETS,test brand @1612366146827,False,601ac142be37ce2ead43755e,5332fa12e4b03c9a25efd1e7,Cogs,TEST BRANDCODE @1612366146827


In [42]:
#Renaming column headers to user-friendly names

brandcols = {'_id.$oid':'brand_id',
             'cpg.$id.$oid':'cpg_id',
             'cpg.$ref':'cpf_ref'}

df_brands.rename(columns= brandcols, inplace=True)

df_brands.columns

Index(['barcode', 'category', 'categoryCode', 'name', 'topBrand', 'brand_id',
       'cpg_id', 'cpf_ref', 'brandCode'],
      dtype='object')

Normalizing and reading users table


In [33]:
# Read the JSON file with one JSON object per line
with open('/content/drive/MyDrive/Fetch Sample Data/users.json') as f:
    lines = f.readlines()

# Normalize each JSON object and convert date columns
normalized_data = []
for line in lines:
    data = json.loads(line)
    df_users = pd.json_normalize(data)

    # Convert specific date columns to datetime format
    date_columns = ['createdDate.$date', 'lastLogin.$date']
    for column in date_columns:
        if column in df_users.columns:
            df_users[column] = pd.to_datetime(int(df_users[column])/1000, unit='s')

    normalized_data.append(df_users)

# Concatenate all normalized DataFrames
df_users = pd.concat(normalized_data, ignore_index=True)

# Print the resulting DataFrame
df_users.head(5)

,active,role,signUpSource,state,_id.$oid,createdDate.$date,lastLogin.$date
0,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04.800000000,2021-01-03 15:25:37.858000128
1,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04.800000000,2021-01-03 15:25:37.858000128
2,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04.800000000,2021-01-03 15:25:37.858000128
3,True,consumer,Email,WI,5ff1e1eacfcf6c399c274ae6,2021-01-03 15:25:30.553999872,2021-01-03 15:25:30.596999936
4,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04.800000000,2021-01-03 15:25:37.858000128


In [34]:
#Renaming column headers to user-friendly names

usercols = {'_id.$oid':'user_id',
             'createdDate.$date':'createdDate',
             'lastLogin.$date':'lastLogin'}

df_users.rename(columns= usercols, inplace=True)

df_users.columns
df_users.head(5)

,active,role,signUpSource,state,user_id,createdDate,lastLogin
0,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04.800000000,2021-01-03 15:25:37.858000128
1,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04.800000000,2021-01-03 15:25:37.858000128
2,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04.800000000,2021-01-03 15:25:37.858000128
3,True,consumer,Email,WI,5ff1e1eacfcf6c399c274ae6,2021-01-03 15:25:30.553999872,2021-01-03 15:25:30.596999936
4,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04.800000000,2021-01-03 15:25:37.858000128


# Q2. Writing a SQL Query that directly answers the Stakeholders question

In [35]:
import sqlite3

In [36]:
conn = sqlite3.connect(':memory:')

In [38]:
# Dropping duplicates and loading the users table to SQL

df_users.drop_duplicates(subset=['user_id']).to_sql('users',con=conn)

ValueError: ignored

In [39]:
#Renaming the column header to replace "." to "_" to make it SQL friendly

df_receipts_final.columns = df_receipts_final.columns.str.replace(".", "_")

<ipython-input-39-1f003eed476e>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_receipts_final.columns = df_receipts_final.columns.str.replace(".", "_")


In [40]:
#Dropping the original rewardsReceiptItemList column and loading the Receipts table to SQL

df_receipts_final.drop(columns=['rewardsReceiptItemList'],axis=1).to_sql('receipts',con=conn)

7381

In [44]:
#Dropping duplicates and loading the brands table to SQL

df_brands.drop_duplicates(subset=['brand_id']).to_sql('brands',con=conn)

1167

In [60]:
#SQL Query to find whether average spend is higher for rewardsReceiptStatus of 'Accepted' or 'Rejected'

query = '''
SELECT "Finished" AS Status, avg(totalSpent) from receipts
WHERE rewardsReceiptStatus in ("FINISHED")
UNION
SELECT "Rejected", avg(totalSpent) from receipts
WHERE rewardsReceiptStatus = "REJECTED"
'''

In [132]:
# Ranking total spent for all statuses

query = '''
SELECT rewardsReceiptStatus, avg(totalSpent) as TotalSpent from receipts
GROUP BY rewardsReceiptStatus
ORDER BY TotalSpent DESC
'''

In [133]:
# Execute the SQL query using the SQL engine
result = pd.read_sql_query(query, conn)

# Print the resulting DataFrame
print(result)

  rewardsReceiptStatus   TotalSpent
0              FLAGGED  2635.570247
1             FINISHED  1244.372934
2              PENDING    28.032449
3             REJECTED    19.544970
4            SUBMITTED          NaN


We can see that Finshed does have a higher average spent amount than the rejected receipts. It can also be observed that the "Flagged" receipts have the highest average spent value

In [72]:
# Which brand has the most spend among users who were created within the past 6 months?

# Note: For this query, instead of using the current date, the lastest date of the created user from the Users table has been used, since the latest date is in 2021 February

query2 = '''
SELECT
  r.rewardsReceiptItemList_brandCode as BrandCode,
  SUM(r.totalSpent) as TotalSpent
FROM
  receipts r
WHERE
  r.userID
IN
  (
    SELECT user_id from users u where u.createdDate > (SELECT DATE('2021-02-12','-6 months'))
  )
AND r.rewardsReceiptItemList_brandCode NOT NULL
GROUP BY r.rewardsReceiptItemList_brandCode
ORDER BY TotalSpent DESC
LIMIT 1;
'''



In [134]:
# Execute the SQL query using the SQL engine
result2 = pd.read_sql_query(query2, conn)

# Print the resulting DataFrame
print(result2)

        BrandCode  TotalSpent
0  BEN AND JERRYS   197337.68


Within the users created in the last 6 months, we can see that Ben and Jerry's brand has the most spent amount associated with it

In [124]:
# Which brand has the most transactions among users who were created within the past 6 months?

# Note: For this query, instead of using the current date, the lastest date of the created user from the Users table has been used, since the latest date is in 2021 February

query2_1 = '''
SELECT
  r.rewardsReceiptItemList_brandCode as BrandCode,
  COUNT(*) as NumberOfTransactions
FROM
  receipts r
WHERE
  r.userID
IN
  (
    SELECT user_id from users u where u.createdDate > (SELECT DATE('2021-02-12','-6 months'))
  )
AND r.rewardsReceiptItemList_brandCode NOT NULL
GROUP BY r.rewardsReceiptItemList_brandCode
ORDER BY NumberOfTransactions DESC
LIMIT 1;
'''

In [125]:
# Execute the SQL query using the SQL engine
result2 = pd.read_sql_query(query2_1, conn)

# Print the resulting DataFrame
print(result2)

  BrandCode  NumberOfTransactions
0    HY-VEE                   291


For the users created in the last 6 months, HY-VEE brand has the most number of transactions

#Q3. Evaluate Data Quality Issues in the Data Provided

To inspect data quality, I will be checking the completeness of the dimension tables Users and Brands by doing referential integrity checks between the Receipts table and the Users and brands

In [87]:
query3 = '''
SELECT userID from receipts
where userID NOT IN
(SELECT user_id from users)
'''

In [88]:
# Execute the SQL query using the SQL engine
result3 = pd.read_sql_query(query3, conn)

# Print the resulting DataFrame
print(result3)

                        userId
0     5f9c74f7c88c1415cbddb839
1     5f9c74f7c88c1415cbddb839
2     5f9c74f7c88c1415cbddb839
3     5f9c74f7c88c1415cbddb839
4     5f9c74f7c88c1415cbddb839
...                        ...
2605  60253861efa6017a44dc6b50
2606  60253891b54593795bf69242
2607  60253891b54593795bf69242
2608  60268c7bb545931ac63683af
2609  60268c78efa6011bb151077d

[2610 rows x 1 columns]


We can see that there are 2610 userIDs in the receipts table which are not present in the reference Users table. This could be problematic as there will be a large loss of data while joining and querying between the Users and Receipts table

In [79]:
query4 = '''
SELECT count(rewardsReceiptItemList_brandCode) from receipts
where rewardsReceiptItemList_brandCode NOT IN
(SELECT brandCode from brands)
'''

In [80]:
# Execute the SQL query using the SQL engine
result4 = pd.read_sql_query(query4, conn)

# Print the resulting DataFrame
print(result4)

   count(rewardsReceiptItemList_brandCode)
0                                        0


The brandCodes in the receipts table do match with the brandCodes in the brand table, however, there are a lot of brand code records with NULL brandCode, which may need further investigation

In [102]:
#Next, we will be checking if the barcodes in the receipts table match with the barcode field in the brands table

query5 = '''
SELECT count(DISTINCT rewardsReceiptItemList_barcode) from receipts
WHERE rewardsReceiptItemList_barcode NOT IN
(SELECT barcode from brands)'''

In [103]:
result5 = pd.read_sql_query(query5, conn)

# Print the resulting DataFrame
print(result5)

   count(DISTINCT rewardsReceiptItemList_barcode)
0                                             552


In [105]:
query6 = ''' SELECT count(DISTINCT rewardsReceiptItemList_barcode) from receipts'''

In [106]:
result6 = pd.read_sql_query(query6, conn)

# Print the resulting DataFrame
print(result6)

   count(DISTINCT rewardsReceiptItemList_barcode)
0                                             568


Out of the 568 unique barcodes in the receipts table, there are 552 barcodes which do not match with the barcodes in the brand table. Even though the documentation says barcode of the brands table refers to barcode of the item, further clarification might be needed from stakeholers on the source of the different barcodes

Checking for NULLS

In [109]:
query7 = ''' SELECT COUNT(*) from receipts where userID IS NULL'''

result7 = pd.read_sql_query(query7, conn)
print(result7)

   COUNT(*)
0         0


In [110]:
query8 = ''' SELECT COUNT(*) from receipts where rewardsReceiptItemList_barcode IS NULL'''

result8 = pd.read_sql_query(query8, conn)
print(result8)

   COUNT(*)
0      4291


In [111]:
query9 = ''' SELECT COUNT(*) from receipts where rewardsReceiptItemList_brandCode IS NULL'''

result9 = pd.read_sql_query(query9, conn)
print(result9)

   COUNT(*)
0      4781


We can see that a lot of the receipts table entires are NULL for barcodes and brandCodes. It might be problematic as these seem to be the the identifiers for the item which is being billed. Not having a consistent identifier can make it difficult to track the revenues as well as in modeling the data efficiently.